In [ ]:
# ! pip install --upgrade pip
# ! pip install --user numpy scipy matplotlib pyhf iminuit
# ! pip install --user --pre eoshep

# Or follow installation instructions https://eos.github.io/doc/installation.html

In [1]:
import numpy as np
import pyhf
import eos

In [ ]:
model = pyhf.simplemodels.correlated_background(
            signal=[50, 100], bkg=[500, 600], bkg_down=[490, 580], bkg_up=[510, 620]
        )

data = [600, 800]

In [ ]:
pyhf.set_backend("jax", pyhf.optimize.minuit_optimizer())
best_fit = pyhf.infer.mle.fit(data+model.config.auxdata, model, return_uncertainties=True)
best_fit

In [ ]:
import yaml

eos_analysis = {
    'likelihoods': [{
        'name': 'EXP-pyhf',
        'pyhf': {'file': 'workspace.json'}
    }],
    'priors': [{
        'name': 'MU',
        'descriptions': [{ 'parameter': 'pyhf::mu', 'central':  1., 'sigma': 5., 'type': 'gaussian' }]
        # 'descriptions': [{ 'parameter': 'pyhf::mu', 'min':  1., 'max': 3., 'type': 'uniform' }]
    }],
    'posteriors': [{
        'name': 'EXP-pyhf',
        'prior': ['MU'],
        'likelihood': ['EXP-pyhf']
    }]
}

with open('eos_analysis.yaml', 'w') as f:
    yaml.dump(eos_analysis, f)

In [ ]:
af = eos.AnalysisFile("eos_analysis.yaml")
af

In [ ]:
eos.tasks.find_mode(af, 'EXP-pyhf', base_directory='./')

In [ ]:
mode = eos.data.Mode('./EXP-pyhf/mode-default')
mode.mode

In [ ]:
eos.tasks.sample_nested(af, 'EXP-pyhf', base_directory='./', nlive=100, maxiter=3000, dlogz=9.0)

In [ ]:
eos.tasks.corner_plot(af, 'EXP-pyhf')

In [ ]:
parameter_samples = eos.data.ImportanceSamples('./EXP-pyhf/samples')

mean = np.average(parameter_samples.samples[:,0], weights = parameter_samples.weights)
std = np.sqrt(np.average((parameter_samples.samples[:,0]-mean)**2, weights = parameter_samples.weights))

print(f'mu = {mean:.4f} +/- {std:.4f}')